# Projet : Classification du Titanic (Régression logistique)

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # Pour le jeu de données Titanic et la visualisation

# Outils de Scikit-learn
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression # Modèle de classification 
from sklearn import metrics

### Préparation des données

In [ ]:
# Chargement du jeu de données Titanic
data = sns.load_dataset('titanic')

# Définition des features (X) et de la cible (Y)
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
target = 'survived' # 0 = Décédé, 1 = Survivant

# Créer un sous-DataFrame de travail
df = data[features + [target]].copy()

print("\n--- Aperçu du DataFrame de travail ---")
print(df.head())
print("\nStatut des valeurs manquantes avant nettoyage :")
print(df.isnull().sum())


--- Aperçu du DataFrame de travail ---
   pclass     sex   age  sibsp  parch     fare  survived
0       3    male  22.0      1      0   7.2500         0
1       1  female  38.0      1      0  71.2833         1
2       3  female  26.0      0      0   7.9250         1
3       1  female  35.0      1      0  53.1000         1
4       3    male  35.0      0      0   8.0500         0

Statut des valeurs manquantes avant nettoyage :
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
survived      0
dtype: int64


Les variables explicatives utilisées incluent la classe du billet, le sexe, l’âge, la composition familiale à bord et le prix du billet.

La colonne 'age' contient des valeurs manquantes (NaN) que nous devons gérer.
La solution la plus simple est de remplacer les NaN par l'âge médian pour ne pas biaiser le modèle.

In [10]:
median_age = df['age'].median()
df['age'] = df['age'].fillna(median_age)
print(f"\nValeurs manquantes dans 'age' remplacées par la médiane ({median_age:.2f} ans).")

# Supprimer les quelques lignes qui pourraient encore contenir des NaN (comme 'fare')
df = df.dropna()


Valeurs manquantes dans 'age' remplacées par la médiane (28.00 ans).
